---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    train.csv - the training set (all tickets issued 2004-2011)
    test.csv - the test set (all tickets issued 2012-2016)
    addresses.csv & latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

In [ ]:

import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
# pd.set_option('display.max_columns', 10)
def blight_model():
    
    train = pd.read_csv('train.csv', encoding = "ISO-8859-1")
    test = pd.read_csv('test.csv', encoding = "ISO-8859-1")
    add = pd.read_csv('addresses.csv', encoding = "ISO-8859-1")
    lat = pd.read_csv('latlons.csv', encoding = "ISO-8859-1")
    
    final_train = pd.merge(train, add , on='ticket_id', how='inner')
    final_test = pd.merge(test, add , on='ticket_id', how='inner')
    
    
#     final_train = pd.merge(final_train, lat , on='address', how='inner')
#     final_test = pd.merge(final_test, lat , on='address', how='inner')
    
    final_train = final_train[np.isfinite(final_train['compliance'])]
    final_train = final_train.drop(['non_us_str_code', 'country' ,'mailing_address_str_number', 'violation_description',  'address', 'disposition',  'fine_amount',  'judgment_amount','inspector_name','ticket_issued_date','hearing_date' ,'violation_code','state_fee' ,'late_fee' ,'discount_amount' ,'clean_up_cost' ,'admin_fee' ,'mailing_address_str_name' ,'violation_street_name' ,'violator_name',            'violation_zip_code', 'collection_status', 'grafitti_status','payment_amount', 'balance_due', 'payment_date', 'payment_status', 'compliance_detail'], axis=1)
    final_test = final_test.drop(['non_us_str_code', 'country', 'mailing_address_str_number', 'violation_description', 'address' , 'disposition',  'fine_amount',  'judgment_amount','inspector_name','ticket_issued_date','hearing_date' ,'violation_code','state_fee' ,'late_fee' ,'discount_amount' ,'clean_up_cost' ,'admin_fee' ,'mailing_address_str_name' ,'violation_street_name' ,'violator_name',            'violation_zip_code', 'grafitti_status'], axis=1)
    final_train['compliance'] = final_train['compliance'].astype(int)
    
    convert_int ={'ticket_id','violation_street_number','zip_code'}
    convert_cat = {'agency_name', 'zip_code', 'city', 'state'}
    all_col = {'agency_name', 'zip_code', 'city', 'state','ticket_id','violation_street_number','zip_code'}
     
    
    for col in convert_cat :
        final_train[col] = final_train[col].replace(np.nan, "NA", regex=True).astype('category')
        final_test[col] = final_test[col].replace(np.nan, "NA", regex=True).astype('category')
        
    
    
    y_train = final_train['compliance']
    final_train = final_train.drop(['compliance'], axis = 1) 
#     print(final_train.isnull().sum())

    cat_columns = final_train.select_dtypes(['category']).columns
    final_train[cat_columns] = final_train[cat_columns].apply(lambda x: x.cat.codes)
    final_test[cat_columns] = final_test[cat_columns].apply(lambda x: x.cat.codes)
    
    X_test = final_test.set_index('ticket_id')
    final_train = final_train.set_index('ticket_id')
    
#     print(final_train.head(10))
    
#     print(final_test.shape)
    
#    print(y_train.head(10))
    
    
    
    grid_val = {'learning_rate': [0.01, 0.1, 1.0], 'max_depth': [3, 4, 5] }
    model = GradientBoostingClassifier(random_state = 0)
    grid_search = GridSearchCV(model, param_grid = grid_val, scoring = 'roc_auc')
    grid_search.fit(final_train, y_train)
    probilities = grid_search.predict_proba(X_test)[:, 1]
    
 
    
    result = pd.Series(probilities, index=test.index)
    
#     print(final_train)
#     for df in [final_test, final_train]:
#         for col in convert_cat :
#             if col in df:
#                 print(df[col])
                
        
#     features = []
#     for i in range(0, 6):
#         label_encoder = LabelEncoder()
#         feature = label_encoder.fit_transform(X[:,i])
#         features.append(feature)
# encoded_x = numpy.array(features)
# encoded_x = encoded_x.reshape(X.shape[0], X.shape[1]) 
#     one = list(final_train.columns.values)
#     two = list(final_test.columns.values)
#     print(one)
#     print(two)
    
#     trainx = final_train.drop(train_cols_to_drop, axis=1)
#     print(trainx.head(10))
#     two = list(trainx.columns.values)
#     col_obj = final_train.select_dtypes(include = ['object'])
    
#     col_num= final_train.select_dtypes(include = [np.number])

#Pre-processing data
#     le = preprocessing.LabelEncoder()
#     col_obj__ = col_obj.apply(le.fit_transform)
#     processed_data = pd.concat([col_obj__,col_num], axis =1)
#     print(one)
#     print(processed_data.head(10))
#     min_max_scaler = preprocessing.MinMaxScaler()
#     x_scaled = min_max_scaler.fit_transform(final_train.columns.intersection())
#     df = pd.DataFrame(x_scaled)
#     #print(final_test.head(20))
#     print(df.corr() )
    
    return result


In [ ]:
blight_model()

/opt/conda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2827: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
